In [11]:
import os, time, json, csv
from datetime import datetime, timedelta
import logging
import re
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [12]:
def log_setup(name, log_file):
    """Setup log for the whole process
    Parameters:
    `log_file` (string): Log file name.
    """
    log_handler = logging.handlers.TimedRotatingFileHandler(
    log_file, when="midnight", backupCount=3)
    formatter = logging.Formatter(
    "%(asctime)s - %(levelname)s - [%(process)d]: %(message)s",
    "%Y-%m-%d %H:%M:%S")
    log_handler.setFormatter(formatter)
    log_obj = logging.getLogger(name)
    log_obj.addHandler(log_handler)
    log_obj.setLevel(logging.INFO)
    
    return log_obj

logFile = log_setup('logFile', "log/crawling-gmaps.log")

In [13]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")
# chrome_options.add_argument("--headless")

In [14]:
chrome_install = ChromeDriverManager().install()

folder = os.path.dirname(chrome_install)
chromedriver_path = os.path.join(folder, "chromedriver.exe")

desired_capabilities = DesiredCapabilities.CHROME
driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)
driver.maximize_window()

C:\Users\User\AppData\Local\Temp\ipykernel_9668\4267024452.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)


In [15]:
driver.get("https://www.google.com/maps/?hl=en")

In [16]:
find_poi = ['INFORMA', 'AZKO', 'MR.DIY', 'MINISO', 'NICESO']
f_city_ind = 444
l_city_ind = 448

In [17]:
with open('list_city.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
list_city = [data[idx][0] if (idx >= f_city_ind) and (idx <= l_city_ind) else None for idx in range(len(data))]
list_city = [city for city in list_city if city is not None]
list_city

['KOTA JAKARTA BARAT DKI JAKARTA',
 'KOTA JAKARTA PUSAT DKI JAKARTA',
 'KOTA JAKARTA SELATAN DKI JAKARTA',
 'KOTA JAKARTA TIMUR DKI JAKARTA',
 'KOTA JAKARTA UTARA DKI JAKARTA']

In [18]:
for fp in find_poi:
    for city in list_city:
        logFile.info(f'Start Crawling for POI: {fp}')
        resultFile = log_setup('resultFile', f'result/data_gmaps_poi_{fp.replace(" ","_")}_{f_city_ind}_{l_city_ind}_{datetime.now().strftime("%Y%m")}.txt')

In [19]:
for fp in find_poi:
    file_done_city = f'done/list_city_done_{fp.replace(" ","_")}_{f_city_ind}_{l_city_ind}_{datetime.now().strftime("%Y%m")}.csv'
    if os.path.exists(file_done_city):
        with open(file_done_city, newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
        list_done_city = [x[0] for x in data]
    else:
        list_done_city = []
list_done_city

['KOTA JAKARTA BARAT DKI JAKARTA',
 'KOTA JAKARTA PUSAT DKI JAKARTA',
 'KOTA JAKARTA SELATAN DKI JAKARTA',
 'KOTA JAKARTA TIMUR DKI JAKARTA',
 'KOTA JAKARTA UTARA DKI JAKARTA',
 'KOTA JAKARTA BARAT DKI JAKARTA',
 'KOTA JAKARTA PUSAT DKI JAKARTA',
 'KOTA JAKARTA SELATAN DKI JAKARTA',
 'KOTA JAKARTA TIMUR DKI JAKARTA',
 'KOTA JAKARTA UTARA DKI JAKARTA',
 'KOTA JAKARTA BARAT DKI JAKARTA',
 'KOTA JAKARTA PUSAT DKI JAKARTA',
 'KOTA JAKARTA SELATAN DKI JAKARTA',
 'KOTA JAKARTA TIMUR DKI JAKARTA',
 'KOTA JAKARTA UTARA DKI JAKARTA',
 'KOTA JAKARTA BARAT DKI JAKARTA',
 'KOTA JAKARTA PUSAT DKI JAKARTA',
 'KOTA JAKARTA SELATAN DKI JAKARTA',
 'KOTA JAKARTA TIMUR DKI JAKARTA',
 'KOTA JAKARTA UTARA DKI JAKARTA',
 'KOTA JAKARTA BARAT DKI JAKARTA',
 'KOTA JAKARTA PUSAT DKI JAKARTA',
 'KOTA JAKARTA SELATAN DKI JAKARTA',
 'KOTA JAKARTA TIMUR DKI JAKARTA',
 'KOTA JAKARTA UTARA DKI JAKARTA']

In [20]:
provinsi = 'DKI JAKARTA'
csv_header = ['Provinsi', 'Kabupaten/Kota', 'Kecamatan', 'Nama', 'Kategori', 'Latitude', 'Longitude', 'Alamat', 'Link']

for fp in find_poi:
    for city in list_city:
        kab_kota_clean = re.sub(r'\s*DKI JAKARTA$', '', city).strip()
        csv_filename = f"{fp}_{provinsi}.csv"
        write_header = not os.path.exists(csv_filename)

        logFile.info(f'Start Crawling for POI: {fp} in {kab_kota_clean}')
        try:
            searchbox = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
            searchbox.clear()
            searchbox.send_keys(f"{fp} {kab_kota_clean}")
            time.sleep(2)
            searchbox.send_keys(Keys.RETURN)
            time.sleep(2)

            result_container_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'
            result_el = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, result_container_xpath))
            )

            elements = result_el.find_elements(By.CLASS_NAME, 'hfpxzc')
            last_count = len(elements)
            try_cnt = 5
            while True:
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight + 5000", result_el)
                time.sleep(2)
                elements = result_el.find_elements(By.CLASS_NAME, 'hfpxzc')
                current_count = len(elements)
                if current_count == last_count:
                    try_cnt -= 1
                    if try_cnt < 0:
                        break
                else:
                    last_count = current_count
                    try_cnt = 5

            logFile.info(f"{fp} in {city} Total Data: {last_count}")
            print(f"{fp} in {city} Total Data: {last_count}")

        except Exception as e:
            logFile.error(f"Error occurred while crawling {fp} in {kab_kota_clean}: {e}")
            continue

        idx = 1
        
        elements = result_el.find_elements(By.CLASS_NAME, 'hfpxzc')
        total_elements = len(elements)

        for i in range(total_elements):
            try:
                elements = result_el.find_elements(By.CLASS_NAME, 'hfpxzc')
                if i >= len(elements):
                    logFile.warning(f"Element index {i} not found after refresh.")
                    continue
                
                el = elements[i]

                driver.execute_script("arguments[0].scrollIntoView();", el)
                time.sleep(2)
                try:
                    el.click()
                except:
                    driver.execute_script("arguments[0].click();", el)  # Fallback dengan JS
                time.sleep(2)

                data = {}
                data['city'] = city
                try:
                    name_el = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '//h1[contains(@class, "DUwDvf") or contains(@class, "fontHeadlineLarge")]'))
                    )
                    data['name'] = name_el.text.strip()
                except:
                    data['name'] = 'N/A'

                try:
                    alamat_el = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "Io6YTe")]'))
                    )
                    data['alamat'] = alamat_el.text
                    match = re.search(r'(?:Kec\.?|Kecamatan)\s+([A-Za-z\s\.\-]+)', data['alamat'])
                    if match:
                        kec = match.group(1).strip()
                        kec = kec.replace('.', '')  
                        data['kec'] = kec
                    else:
                        data['kec'] = "Unknown"
                except:
                    data['alamat'] = "Unknown"
                    data['kec'] = "Unknown"

                try:
                    cat_el = driver.find_element(By.XPATH, '//button[contains(@class, "DkEaL")]')
                    data['cat'] = cat_el.text
                except:
                    data['cat'] = "Unknown"

                try:
                    share_btn = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '//button[@data-value][@aria-label]'))
                    )
                    data['link'] = driver.current_url
                except:
                    data['link'] = driver.current_url

                dt_href = data['link'].split("/place/")[1].split("!4m")[0]
                match_latlon = re.findall(r'@(-?\d+\.\d+),(-?\d+\.\d+)', data['link'])
                if match_latlon:
                    data['lat'], data['lon'] = match_latlon[0]
                else:
                    data['lat'], data['lon'] = '', ''

                data["crawling_date"] = datetime.now().strftime("%d-%m-%Y %H:%M:%S")

                log_line = f"{idx}. {data['name']} - {data['cat']} - {data['alamat']} - {data['kec']} - {data['lat']} - {data['lon']} - {data['link']}"
                print(log_line)
                logFile.info(log_line)
                resultFile.info(f"result##{json.dumps(data)}")

                with open(csv_filename, mode='a', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    if write_header:
                        writer.writerow(csv_header)
                        write_header = False
                    writer.writerow([
                        provinsi,
                        kab_kota_clean,
                        data['kec'],
                        data['name'],
                        data['cat'],
                        data['lat'],
                        data['lon'],
                        data['alamat'],
                        data['link']
                    ])
                idx += 1

                try:
                    close_btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((
                            By.XPATH,
                            '//button[.//svg[path[contains(@d,"M19 6.41L17.59 5")]]]'
                        ))
                    )
                    close_btn.click()
                    time.sleep(2)
                except Exception as e:
                    logFile.warning(f"Gagal klik tombol tutup: {e}")
                    pass

            except Exception as e:
                logFile.error(f"Failed to extract or write data for one element in {city}: {e}")
                continue

        list_done_city.append(city)
        with open(file_done_city, 'w+', encoding='utf-8') as f:
            f.write("\n".join(list_done_city))

INFORMA in KOTA JAKARTA BARAT DKI JAKARTA Total Data: 4
1. INFORMA - LIVING PLAZA PX PURI - Millwork shop - Puri Harum No.2 3, RT.3/RW.2, SELATAN, Kembangan Sel., Kec. Kembangan, Kota Jakarta Barat, Daerah Khusus Ibukota Jakarta 11610 - Kembangan - -6.1886 - 106.6998484 - https://www.google.com/maps/place/INFORMA+-+LIVING+PLAZA+PX+PURI/@-6.1886,106.6998484,14z/data=!4m10!1m2!2m1!1sINFORMA+KOTA+JAKARTA+BARAT!3m6!1s0x2e69f712246658db:0x72a97ad881fa6185!8m2!3d-6.1886!4d106.7379572!15sChpJTkZPUk1BIEtPVEEgSkFLQVJUQSBCQVJBVFocIhppbmZvcm1hIGtvdGEgamFrYXJ0YSBiYXJhdJIBDW1pbGx3b3JrX3Nob3CqAV4KDC9nLzExaDBsdGQxZhABKgsiB2luZm9ybWEoADIfEAEiGwF-b7yPgkLe1BdyIxww3e4NIlGL4R43kpEcPzIeEAIiGmluZm9ybWEga290YSBqYWthcnRhIGJhcmF04AEA!16s%2Fg%2F11h0ltd1f?hl=en&entry=ttu&g_ep=EgoyMDI1MDYwMS4wIKXMDSoASAFQAw%3D%3D
2. INFORMA NEO SOHO - Millwork shop - Neo Soho Mall, Jl. Letjen S. Parman No.Kav 28, Tj. Duren Sel., Kec. Grogol petamburan, Kota Jakarta Barat, Daerah Khusus Ibukota Jakarta 11470 - Grogol petamburan - 